In [ ]:
## MATH AND DATA PROCESSING
import pandas as pd
import numpy as np
from scipy.special import gammaln
from math import lgamma, log
from sklearn.preprocessing import OrdinalEncoder

## PLOT
import matplotlib.pyplot as plt
import networkx as nx

## OS and sys
import sys
import os
import warnings
from itertools import product, chain

In [133]:
from pgmpy.estimators import HillClimbSearch, BicScore, PC, ParameterEstimator
from pgmpy.estimators import BayesianEstimator as BE
from pgmpy.estimators import MaximumLikelihoodEstimator as MLE
from pgmpy.estimators import ExpectationMaximization as EM

from pgmpy.models import BayesianNetwork

from pgmpy.factors.discrete import TabularCPD
from pgmpy.global_vars import SHOW_PROGRESS

from tqdm.auto import tqdm
from pgmpy.base import DAG

In [ ]:
%run ML_score.py
%run EM_modified.ipynb

In [1]:
class PreprocessDataParameter:
    def __init__(self, NODE_SIZE, NUM_BINS, NUM_BINS_SMALL):
        self.NODE_SIZE = NODE_SIZE
        self.NUM_BINS = NUM_BINS
        self.NUM_BINS_SMALL = NUM_BINS_SMALL
        
    def plot(self, model):
        plt.figure(figsize=(8, 6), dpi=100)  
        
        nx.draw(model, with_labels=True, node_size = self.node_sizes)
        
        plt.axis('off')
        axis = plt.gca()
        axis.set_xlim([1.2*x for x in axis.get_xlim()])
        axis.set_ylim([1.2*y for y in axis.get_ylim()])
        
        return plt.show()

    def get_disc_data(self, disc_data):

        teplota = self.cutting_cond(disc_data['Teplota vzduchu'])
        rychlost = self.cutting_cond(disc_data['Rychlost otáček'])
        krout = self.cutting_cond(disc_data['Kroutící moment'])
        opo = self.cutting_cond(disc_data['Opotřebení nástroje'])
        
        return disc_data, teplota, rychlost, krout, opo
    
    @staticmethod
    def cutting_cond(x):
        return (min(x), max(x), (max(x) - min(x)) / 40)
    
    @staticmethod
    def cutting_cond_small(x, bins):
        return (min(x) - 100, max(x) + 100, ((max(x) + 100) - (min(x) - 100)) / bins)
    
    def bin_data(self, x_test, data_to_be_replaced = None):
        NUM_BINS = self.NUM_BINS
        NUM_BINS_SMALL = self.NUM_BINS_SMALL
        
        
        disc_data = x_test.copy()
        disc_data, teplota, rychlost, krout, opo = self.get_disc_data(disc_data)
        
        teplota = self.cutting_cond(disc_data['Teplota vzduchu'], NUM_BINS)
        rychlost = self.cutting_cond(disc_data['Rychlost otáček'], NUM_BINS)
        krout = self.cutting_cond(disc_data['Kroutící moment'], NUM_BINS)
        opo = self.cutting_cond_small(disc_data['Opotřebení nástroje'], NUM_BINS_SMALL)
        
        disc_data['T'] = pd.cut(x = disc_data['Teplota vzduchu'], bins = np.arange(teplota[0], teplota[1], teplota[2]), labels = np.arange(0, NUM_BINS - 1, 1))
        disc_data['R'] = pd.cut(x = disc_data['Rychlost otáček'], bins = np.arange(rychlost[0], rychlost[1], rychlost[2]), labels = np.arange(0, NUM_BINS - 1, 1))
        disc_data['K'] = pd.cut(x = disc_data['Kroutící moment'], bins = np.arange(krout[0], krout[1], krout[2]), labels = np.arange(0, NUM_BINS - 1, 1))
        disc_data['O'] = pd.cut(x = disc_data['Opotřebení nástroje'], bins = np.arange(opo[0], opo[1], opo[2]), labels = np.arange(0, NUM_BINS_SMALL - 1, 1))

        y_test = disc_data['Porucha']
        disc_data = disc_data.dropna(how = 'any')
        
        return disc_data
    

    

In [134]:
enc = OrdinalEncoder()
df = pd.read_csv('gs://bayesian_net/predictive_maintenance.csv')

df = df.drop(columns = ['Unnamed: 0', 'Type', 'Process temperature [K]'])
df.columns = ['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']

df[df['Porucha'] == 1].reset_index(drop = True) 

from sklearn.model_selection import train_test_split

_, x_test, _, y_test = train_test_split(
                                                     df.loc[:, df.columns != 'b'], 
                                                     df['Porucha'], 
                                                     test_size=0.9999, 
                                                     random_state=42
                                                   )

x_test['Porucha'] = y_test

x_test['Teplota vzduchu'] = x_test['Teplota vzduchu'].apply(lambda x: np.float32(x))
x_test['Kroutící moment'] = x_test['Kroutící moment'].apply(lambda x: np.float32(x))
x_test['Rychlost otáček'] = x_test['Rychlost otáček'].apply(lambda x: np.int32(x))
x_test['Opotřebení nástroje'] = x_test['Opotřebení nástroje'].apply(lambda x: np.int32(x))
x_test['Porucha'] = x_test['Porucha'].apply(lambda x: np.int32(x))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 6252 to 860
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Teplota vzduchu      9999 non-null   float32
 1   Rychlost otáček      9999 non-null   int32  
 2   Kroutící moment      9999 non-null   float32
 3   Opotřebení nástroje  9999 non-null   int32  
 4   Porucha              9999 non-null   int32  
dtypes: float32(2), int32(3)
memory usage: 273.4 KB


## Structure Learning

In [154]:

gs = HillClimbSearch(data_skewed_hc[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']])
ml_model = gs.estimate(scoring_method = MLScore(data_normal_pearson[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']]), max_iter = 150, show_progress = True)

ml_model.edges()


  0%|          | 0/150 [00:00<?, ?it/s]

OutEdgeView([('Teplota vzduchu', 'Porucha'), ('Porucha', 'Kroutící moment'), ('Porucha', 'Opotřebení nástroje'), ('Porucha', 'Rychlost otáček')])

In [155]:


gs = HillClimbSearch(data_normal_hc[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']])
ml_model = gs.estimate(scoring_method = MLScore(data_normal_pearson[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']]), max_iter = 150, show_progress = True)


ml_model.edges()


  0%|          | 0/150 [00:00<?, ?it/s]

OutEdgeView([('Teplota vzduchu', 'Porucha'), ('Porucha', 'Kroutící moment'), ('Porucha', 'Opotřebení nástroje'), ('Porucha', 'Rychlost otáček')])

In [149]:


gs = HillClimbSearch(data_normal_hc[['T', 'O', 'K', 'R', 'Porucha']])
ml_model = gs.estimate(scoring_method =BicScore(data_normal_pearson[['T', 'O', 'K', 'R', 'Porucha']]), max_iter = 150, show_progress = True)


ml_model.edges()


  0%|          | 0/150 [00:00<?, ?it/s]

OutEdgeView([('K', 'R'), ('K', 'Porucha'), ('Porucha', 'O'), ('Porucha', 'T')])

In [156]:
pc = PC(data=data_normal_pearson[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']])
pc_model_peaerson = pc.estimate(ci_test = 'pearsonr', significance_level=0.05)

pc_model_peaerson.edges()

  0%|          | 0/5 [00:00<?, ?it/s]

OutEdgeView([('Teplota vzduchu', 'Rychlost otáček'), ('Opotřebení nástroje', 'Porucha')])

In [158]:
pc = PC(data=data_normal_pearson[['Teplota vzduchu', 'Rychlost otáček', 'Kroutící moment', 'Opotřebení nástroje', 'Porucha']])
pc_model_peaerson = pc.estimate(ci_test = 'chi_square', significance_level=0.05)

pc_model_peaerson.edges()

  0%|          | 0/5 [00:00<?, ?it/s]

OutEdgeView([('Kroutící moment', 'Rychlost otáček'), ('Teplota vzduchu', 'Rychlost otáček')])

In [138]:
gs = HillClimbSearch(x_test)
ml_model = gs.estimate(scoring_method = BicScore(x_test), max_iter = 15)

ml_model.edges()


  0%|          | 0/15 [00:00<?, ?it/s]

OutEdgeView([])

In [ ]:
gs = HillClimbSearch(x_test)
ml_model = gs.estimate(scoring_method = BicScore(x_test), max_iter = 8)

In [ ]:
ml_model.edges()

#### pc = PC(data=x_test)
pc_model_chi = pc.estimate(ci_test = 'chi_square', significance_level=0.5)

pc_model_chi.edges()

In [ ]:
estimator = EM(BayesianNetwork(ml_model), x_test)

est = estimator.get_parameters(max_iter = 3, seed = 20000000)

est

In [355]:
pc = PC(data=x_test)
pc_model_peaerson = pc.estimate(ci_test = 'pearsonr', significance_level=0.15)

pc_model_peaerson.edges()

  0%|          | 0/5 [00:00<?, ?it/s]

OutEdgeView([('Opotřebení nástroje', 'Porucha'), ('Teplota vzduchu', 'Rychlost otáček')])

In [ ]:
pc = PC(data=x_test)
pc_model_chi = pc.estimate(ci_test = 'chi_square', significance_level=0.15)

pc_model_chi.edges()

In [ ]:
#### pc = PC(data=x_test)
pc_model_chi = pc.estimate(ci_test = 'chi_square', significance_level=0.05)

pc_model_chi.edges()

In [17]:
def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup
    



In [356]:
model = BayesianNetwork([('O', 'Porucha'), ('T', 'R')])

## Parameter learning

## EM

### EM pearson

In [187]:
model = BayesianNetwork([('O', 'Porucha'), ('T', 'R')])

estimator = EM(BayesianNetwork(model), pearson_data[['T', 'R', 'K', 'O', 'Porucha']])

est_disc = estimator.get_parameters(max_iter = 5, seed = 0)
print_full(est_disc['cpds'][1])

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+----------------------+---------------------+----------------------+----------------------+----------------------+----------------------+--------------------+----------------------+---------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+----------------------+----------------------+--------------------+---------------------+---------------------+---------------------+---------------------+--------------------+--------------------+-------+
| O          | O(10)                | O(11)               | O(12)                | O(13)                | O(14)                | O(15)                | O(16)             

### EM hill climbing

In [188]:
dag = DAG([('R', 'O'), ('R', 'T'), ('R', 'Porucha'), ('K', 'R'), ('K', 'O'), ('K', 'T'), ('O', 'T'), ('O', 'Porucha')])


model = BayesianNetwork([('K', 'Porucha'), ('Porucha', 'O'), ('Porucha', 'R'), ('Porucha', 'T')])

estimator = EM(BayesianNetwork(model), hc_data[['K', 'T', 'R', 'O', 'Porucha']])

est_disc = estimator.get_parameters(max_iter = 5, seed = 0)

print_full(est_disc['cpds'][1])

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+------+------+--------------------+---------------------+----------------------+----------------------+--------------------+----------------------+----------------------+-----------------------+----------------------+---------------------+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+--------------------+---------------------+-------+-------+-------+
| K          | K(2) | K(3) | K(4)               | K(5)                | K(6)                 | K(7)                 | K(8)               | K(9)                 | K(10)                | K(11)                

## Bayesian approach

### Bayes hill climbing

In [189]:


g = DAG()
dag = DAG([('K', 'Porucha'), ('Porucha', 'O'), ('Porucha', 'R'), ('Porucha', 'T')])

estimator = BE(BayesianNetwork(dag), hc_data[['T', 'R', 'K', 'O', 'Porucha']])

est = estimator.get_parameters(prior_type='BDeu', equivalent_sample_size=5)

estimator.estimate_cpd('Porucha')

print_full(estimator.estimate_cpd('Porucha'))

+------------+---------------------+---------------------+--------------------+---------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----------------------+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+---------------------+---------------------+---------------------+--------------------+--------------------+---------------------+---------------------+----------------------+----------------------+
| K          | K(2)                | K(3)                | K(4)               | K(5)                | K(6)                 | K(7)                 | K(8)                 | K(9)                 | K(10)                | K(11)                 | K(12)                | K(13)                | K(14)                | K(15)                | K(16)               | K(17)                | K(18)               | K(19)               | K(

### Bayes pearson

In [192]:


model = BayesianNetwork([('O', 'Porucha'), ('T', 'R')])

estimator = BE(model, pearson_data[['T', 'R', 'K', 'O', 'Porucha']])
est = estimator.get_parameters(prior_type='BDeu', equivalent_sample_size=5)

estimator.estimate_cpd('Porucha')

print_full(estimator.estimate_cpd('Porucha'))

+------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+---------------------+----------------------+---------------------+---------------------+--------------------+-------------------+---------------------+----------------------+----------------------+---------------------+---------------------+----------------------+----------------------+---------------------+---------------------+--------------------+--------------------+---------------------+---------------------+---------------------+
| O          | O(10)              | O(11)                | O(12)                | O(13)                | O(14)                | O(15)                | O(16)               | O(17)                | O(18)               | O(19)               | O(20)              | O(21)             | O(22)               | O(23)                | O(24)                | O(25)               | O(26)               | O(27)           

In [191]:

g = DAG()
dag = DAG([('O', 'Porucha'), ('T', 'R')])

estimator = BE(BayesianNetwork(dag), disc_data[['T', 'R', 'K', 'O', 'Porucha']])

est = estimator.get_parameters(prior_type='BDeu', equivalent_sample_size=5)

print_full(estimator.estimate_cpd('Porucha'))

+------------+----------------------+----------------------+---------------------+----------------------+---------------------+--------------------+
| O          | O(1)                 | O(2)                 | O(3)                | O(4)                 | O(5)                | O(6)               |
+------------+----------------------+----------------------+---------------------+----------------------+---------------------+--------------------+
| Porucha(0) | 0.9746789435425249   | 0.9780806838049603   | 0.9771371769383699  | 0.9794379931994611   | 0.9046968869470234  | 0.702247191011236  |
+------------+----------------------+----------------------+---------------------+----------------------+---------------------+--------------------+
| Porucha(1) | 0.025321056457475167 | 0.021919316195039826 | 0.02286282306163022 | 0.020562006800538914 | 0.09530311305297649 | 0.2977528089887641 |
+------------+----------------------+----------------------+---------------------+----------------------+-

# SEM

In [ ]:

data_normal_pearson = preprocess_pearson.sem_bin_data(20, 25, x_test, pd.read_csv("pred_normal.csv", header = None))
data_normal_hc = preprocess_hc.sem_bin_data(20, 8, x_test, pd.read_csv("pred_normal.csv", header = None))

data_skewed_pearson = preprocess_pearson.sem_bin_data(20, 25, x_test, pd.read_csv("pred_skewed_normal.csv", header = None))

data_skewed_hc = preprocess_hc.sem_bin_data(20, 8, x_test, pd.read_csv("pred_skewed_normal.csv", header = None))

In [129]:



for i in [data_skewed_pearson, data_normal_pearson]:
    g = DAG()
    dag = DAG([('O', 'Porucha'), ('T', 'R')])

    estimator = BE(BayesianNetwork(dag), i[['T', 'R', 'K', 'O', 'Porucha']])

    est = estimator.get_parameters(prior_type='BDeu', equivalent_sample_size=5)

    estimator.estimate_cpd('Porucha')

    print_full(estimator.estimate_cpd('Porucha'))

+------------+----------------------+---------------------+----------------------+----------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+---------------------+---------------------+--------------------+---------+
| O          | O(5)                 | O(6)                | O(7)                 | O(8)                 | O(9)               | O(10)                | O(11)                | O(12)                | O(13)                | O(14)                | O(15)                | O(16)               | O(17)               | O(18)              | O(19)   |
+------------+----------------------+---------------------+----------------------+----------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+---------------------+---------------------+------------

In [124]:


for i in [data_skewed_hc, data_normal_hc]:
    g = DAG()
    dag = DAG([('R', 'O'), ('R', 'T'), ('R', 'Porucha'), ('K', 'R'), ('K', 'O'), ('K', 'T'), ('O', 'T'), ('O', 'Porucha')])

    estimator = BE(BayesianNetwork(dag), i[['T', 'R', 'K', 'O', 'Porucha']])

    est = estimator.get_parameters(prior_type='BDeu', equivalent_sample_size=5)

    estimator.estimate_cpd('Porucha')

    print_full(estimator.estimate_cpd('Porucha'))

+------------+------+---------------------+---------------------+------------------------+-----------------------+------------------------+------------------------+-----------------------+-----------------------+-----------------------+----------------------+----------------------+----------------------+-------+---------------------+-------+-------+-------+-------+-----------------------+---------------------+----------------------+-----------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+----------------------+-----------------------+---------------------+---------------------+---------------------+-------+---------------------+--------------------+----------------------+---------------------+---------------------+---------------------+----------------------+-----------------------+----------------------+----------------------+-----------------------+----------------------

In [131]:

for i in [data_normal_pearson, data_skewed_pearson]:
    model = BayesianNetwork([('O', 'Porucha'), ('T', 'R')])

    estimator = EM(BayesianNetwork(model), i[['T', 'R', 'K', 'O', 'Porucha']])

    est_disc = estimator.get_parameters(max_iter = 5, seed = 0)
    print_full(est_disc['cpds'][1])

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+----------------------+---------------------+--------------------+----------------------+----------------------+---------------------+----------------------+----------------------+---------------------+--------------------+----------------------+--------------------+---------------------+--------------------+-------+
| O          | O(5)                 | O(6)                | O(7)               | O(8)                 | O(9)                 | O(10)               | O(11)                | O(12)                | O(13)               | O(14)              | O(15)                | O(16)              | O(17)               | O(18)              | O(19) |
+------------+----------------------+---------------------+--------------------+------

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+----------------------+---------------------+--------------------+----------------------+----------------------+---------------------+----------------------+----------------------+---------------------+--------------------+----------------------+--------------------+---------------------+--------------------+-------+
| O          | O(5)                 | O(6)                | O(7)               | O(8)                 | O(9)                 | O(10)               | O(11)                | O(12)                | O(13)               | O(14)              | O(15)                | O(16)              | O(17)               | O(18)              | O(19) |
+------------+----------------------+---------------------+--------------------+------

In [132]:

for i in [data_skewed_hc, data_normal_hc]:
    model = BayesianNetwork([('R', 'Porucha'), ('O', 'Porucha')])

    estimator = EM(BayesianNetwork(model), i[['T', 'R', 'K', 'O', 'Porucha']])

    est_disc = estimator.get_parameters(max_iter = 5, seed = 0)
    print_full(est_disc['cpds'][1])

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+------+---------------------+---------------------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+---------------------+---------------------+----------------------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+---------------------+-------+------+---------------------+---------------------+----------------------+------+------+---------------------+------+------+------+-------+---------------------+-------+-------+-------+-------+-------+---------------------+-------+---------------------+---------------------+---------------------+----------------------+------+-----------------------+----------------------+------+------+------+-------+-------+-

  0%|          | 0/5 [00:00<?, ?it/s]

0
1
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
1
2
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
<class 'pgmpy.factors.discrete.CPD.TabularCPD'>
+------------+------+---------------------+---------------------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+---------------------+---------------------+----------------------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+---------------------+-------+------+---------------------+---------------------+----------------------+----------------------+------+------+------+------+------+-------+---------------------+-------+-------+-------+-------+-------+---------------------+-------+---------------------+---------------------+---------------------+----------------------+------+----------------------+---------------------+------+------+------+-------+-------+--